# Example: Reduce Image Objects to a Simple Geometric Shape

`pyspatialfield` can be used to find image features and reduce the features to simple geometric objects, such as the ellipse. 

In this example, a lake in Scotland is located in a satellite image, and the lake is described with its geometric parameters. 

In [1]:
from pyspatialfield.field.featurefield import FeatureField, EllipseFeature
from explorer_util.datasource import DataSource, DataDecoder, base64_decode
from explorer_util.visualization import plot_image_with_annotation
import numpy as np
import matplotlib.pyplot as plt

# Link to a base64-encoded satellite image of a lake in Scotland.
url_source = "https://gist.githubusercontent.com/rtrollebo/e20ec7838549a92abbeedfb4c3eac22c/raw/b588bd0263bb9221b6f1e86575ae9a3be0ee342b/sample-image-loch-ness"

# Load the image, and setup the image decoder.
img_decoder = DataDecoder(DataSource.fetch_from_online_source(url_source))

# Generate image feature map. 
img, img_map = FeatureField.generate_from_single_image(
    img_decoder,                               # Image decoder containing the raw data of the image.
    170,                                       # Threshold
    decoder_callback=base64_decode,            # Attach base64 decoder as callback.
    preprocess=lambda x: 255-x,                # Invert image, to easily threshhold-detect the dark lake.
) 

# Find the feature of interest by inspection
feature_iter = FeatureField.get_features_by_size(img_map)
next(feature_iter)
label_of_interest, _ = next(feature_iter) 

# Highlight the feature of interest, create a FeatureField, and obtain key geometric parameters. 
boolmap = FeatureField.get_feature_from_labelmap(img_map, label_of_interest)
field = FeatureField(boolmap)

# Convert the generic FeatureField to the specific geometric ellipsis shape.
ellipsis = EllipseFeature()
ellipsis.from_feature_field(field)

                                     

This particular lake is a long and narrow one, thus its enclosed ellipse is expected to have semi-major axis (a) much bigger than its semi-minor axis (b), or $a / b >> 1$

In [2]:
print(ellipsis.semi_major_axis / ellipsis.semi_minor_axis)

16.67085123836639


We can find the angle in degrees with the vertical:

In [3]:
print(field.degrees)

-35.49623427170941


To verify its location, the centroid is plotted (blue dot) on the feature. 
(run the notebook to see the image plot)

In [4]:
plot_image_with_annotation(img, field.centroid()[1], field.centroid()[0])